In [ ]:
!pip install ultralytics
!pip install opencv-python
!pip install albumentations
!pip install tqdm
!pip install torch torchvision torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.1/922.1 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Check if the dataset is accessible
!ls "/content/drive/MyDrive/blackjack_dataset"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
blackjack


NameError: name 'content' is not defined

In [ ]:
import os
import cv2
import shutil
from tqdm import tqdm
import albumentations as A

# Source folder paths: train folder images and labels
SOURCE_IMAGES = "/content/drive/MyDrive/blackjack_dataset/blackjack/train/images"
SOURCE_LABELS = "/content/drive/MyDrive/blackjack_dataset/blackjack/train/labels"

# Output folder paths: augmented data folder
OUTPUT_IMAGES = "/content/drive/MyDrive/blackjack_dataset/blackjack/augmented_data/images"
OUTPUT_LABELS = "/content/drive/MyDrive/blackjack_dataset/blackjack/augmented_data/labels"

# Create output folders if they don't exist
os.makedirs(OUTPUT_IMAGES, exist_ok=True)
os.makedirs(OUTPUT_LABELS, exist_ok=True)

# Define Albumentations transformation (without check_validity parameter)
transform = A.Compose(
    [
        A.MotionBlur(p=0.3),  # Apply motion blur
        A.GaussNoise(p=0.3),  # Add random noise (var_limit removed)
        A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.5),  # Adjust brightness & contrast
        A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=30, val_shift_limit=20, p=0.5),
        A.Rotate(limit=10, p=0.7, border_mode=cv2.BORDER_CONSTANT),  # Rotation
        A.HorizontalFlip(p=0.5),  # Horizontal flip
        A.VerticalFlip(p=0.3),    # Vertical flip
    ],
    bbox_params=A.BboxParams(
        format='yolo',      # YOLO format: [x_center, y_center, width, height] (normalized)
        label_fields=['class_labels'],
        min_area=0.0,
        min_visibility=0.0
    )
)

# Function to read YOLO labels from a file
def read_yolo_labels(label_path):
    boxes = []
    if not os.path.exists(label_path):
        return boxes  # Return empty list if file doesn't exist
    with open(label_path, "r") as f:
        for line in f.readlines():
            tokens = line.strip().split()
            if len(tokens) < 5:
                continue  # Skip lines with insufficient tokens
            # Use only the first 5 tokens: class, x_center, y_center, width, height
            c, x, y, w, h = tokens[:5]
            boxes.append([float(x), float(y), float(w), float(h), int(c)])
    return boxes

# Function to write YOLO labels to a file
def write_yolo_labels(label_path, bboxes):
    with open(label_path, "w") as f:
        for box in bboxes:
            c, x, y, w, h = box
            f.write(f"{int(c)} {x:.6f} {y:.6f} {w:.6f} {h:.6f}\n")

# Function to clip a YOLO bbox to image boundaries
def clip_bbox_yolo(bbox, img_width, img_height):
    # bbox is [x_center, y_center, width, height] (normalized)
    x_center, y_center, w, h = bbox
    # Convert to absolute coordinates:
    abs_x_center = x_center * img_width
    abs_y_center = y_center * img_height
    abs_w = w * img_width
    abs_h = h * img_height
    # Compute corners:
    x_min = abs_x_center - abs_w / 2
    y_min = abs_y_center - abs_h / 2
    x_max = abs_x_center + abs_w / 2
    y_max = abs_y_center + abs_h / 2
    # Clip coordinates:
    x_min = max(x_min, 0)
    y_min = max(y_min, 0)
    x_max = min(x_max, img_width)
    y_max = min(y_max, img_height)
    # Recompute center and dimensions:
    abs_x_center = (x_min + x_max) / 2
    abs_y_center = (y_min + y_max) / 2
    abs_w = x_max - x_min
    abs_h = y_max - y_min
    # Convert back to normalized coordinates:
    new_x_center = abs_x_center / img_width
    new_y_center = abs_y_center / img_height
    new_w = abs_w / img_width
    new_h = abs_h / img_height
    return [new_x_center, new_y_center, new_w, new_h]

# Main loop: Generate augmentations for each image
num_augs_per_image = 4  # Create 4 augmentations per image

all_images = [f for f in os.listdir(SOURCE_IMAGES) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

for img_name in tqdm(all_images, desc="Augmenting images"):
    img_path = os.path.join(SOURCE_IMAGES, img_name)
    base_name = os.path.splitext(img_name)[0]
    label_path = os.path.join(SOURCE_LABELS, base_name + ".txt")

    # Read the image
    image = cv2.imread(img_path)
    if image is None:
        continue  # Skip if image cannot be read

    # Read YOLO labels
    yolo_bboxes = read_yolo_labels(label_path)
    if len(yolo_bboxes) == 0:
        continue  # Skip if no labels are found

    # Optionally, copy the original image and label to the output folder
    shutil.copy2(img_path, os.path.join(OUTPUT_IMAGES, img_name))
    shutil.copy2(label_path, os.path.join(OUTPUT_LABELS, base_name + ".txt"))

    # Prepare bounding boxes and class labels for augmentation
    bboxes = []
    class_labels = []
    for xb, yb, wb, hb, cls_id in yolo_bboxes:
        bboxes.append([xb, yb, wb, hb])
        class_labels.append(cls_id)

    # Generate multiple augmentations per image
    for i in range(num_augs_per_image):
        # Try the transformation; if a ValueError occurs (due to bbox range), clip input bboxes and retry.
        try:
            transformed = transform(image=image, bboxes=bboxes, class_labels=class_labels)
        except ValueError as e:
            # Clip input bboxes manually before transformation
            bboxes_clipped = [clip_bbox_yolo(bbox, image.shape[1], image.shape[0]) for bbox in bboxes]
            transformed = transform(image=image, bboxes=bboxes_clipped, class_labels=class_labels)

        aug_image = transformed['image']
        aug_bboxes = transformed['bboxes']
        aug_labels = transformed['class_labels']

        if len(aug_bboxes) == 0:
            continue  # Skip if augmentation results in no boxes

        # Get image dimensions
        h_img, w_img, _ = aug_image.shape

        aug_filename = f"{base_name}_aug{i}.jpg"
        aug_labelname = f"{base_name}_aug{i}.txt"
        aug_img_path = os.path.join(OUTPUT_IMAGES, aug_filename)
        aug_lbl_path = os.path.join(OUTPUT_LABELS, aug_labelname)

        cv2.imwrite(aug_img_path, aug_image)

        # Convert bounding boxes to final YOLO format, clipping if needed
        final_bboxes = []
        for bbox, cls_id in zip(aug_bboxes, aug_labels):
            clipped = clip_bbox_yolo(bbox, w_img, h_img)
            final_bboxes.append([cls_id] + clipped)
        write_yolo_labels(aug_lbl_path, final_bboxes)

print("Augmentation process complete!")


Augmenting images: 100%|██████████| 755/755 [06:24<00:00,  1.96it/s]

Augmentation process complete!


In [ ]:
from ultralytics import YOLO
model = YOLO("/content/drive/MyDrive/blackjack_dataset/blackjack/pretrained_model_final.pt")

model.train(
    data="/content/drive/MyDrive/blackjack_dataset/blackjack/data (1).yaml",
    epochs=50,
    imgsz=640,
    device="cuda"
)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.81 🚀 Python-3.11.11 torch-2.5.1+cu124 


ValueError: Invalid CUDA 'device=0' requested. Use 'device=cpu' or pass valid CUDA device(s) if available, i.e. 'device=0' or 'device=0,1,2,3' for Multi-GPU.

torch.cuda.is_available(): False
torch.cuda.device_count(): 0
os.environ['CUDA_VISIBLE_DEVICES']: None
See https://pytorch.org/get-started/locally/ for up-to-date torch install instructions if no CUDA devices are seen by torch.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls -lh runs/detect/train2


ls: cannot access 'runs/detect/train2': No such file or directory


In [ ]:
import subprocess

subprocess.run(["yolo", "train", "model=yolov8m.pt", "data=data.yaml", "epochs=100"])


CompletedProcess(args=['yolo', 'train', 'model=yolov8m.pt', 'data=data.yaml', 'epochs=100'], returncode=1)

In [ ]:
# train_results = model.train(
#     data="/content/drive/MyDrive/blackjack_dataset/blackjack/data.yaml",
#     epochs=100,
#     imgsz=640,
#     rect=True,
#     device="cuda",
#     degrees=10,             # Rotation
#     translate=0.2,          # Perspective shift
#     scale=0.5,              # Scaling
#     shear=10,               # Shear transformation
#     flipud=0.5,             # Vertical flip
#     fliplr=0.5,             # Horizontal flip
#     hsv_h=0.015,            # Hue augmentation
#     hsv_s=0.7,              # Saturation augmentation
#     hsv_v=0.4,              # Brightness augmentation
#     mosaic=1.0,             # Enable mosaic augmentation
#     mixup=0.2               # Mixup augmentation
# )


In [ ]:
# Evaluate model performance on the test dataset defined in your YAML
metrics = model.val(data="/content/drive/MyDrive/blackjack_dataset/blackjack/data.yaml")
print(metrics)


Ultralytics 8.3.78 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 92 layers, 25,872,763 parameters, 0 gradients, 78.9 GFLOPs


val: Scanning /content/drive/MyDrive/blackjack_dataset/blackjack/valid/labels.cache... 74 images, 0 backgrounds, 0 corrupt: 100%|██████████| 74/74 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:05<00:00,  1.14s/it]


                   all         74       1656      0.861      0.747      0.824      0.744
                   10c         19         19      0.925      0.947      0.952      0.836
                   10d         11         11      0.827          1      0.981       0.89
                   10h         16         16      0.918      0.875      0.947      0.852
                   10s         15         15          1       0.92      0.995      0.894
                    2c         16         19      0.689      0.789      0.778      0.647
                    2d         15         19      0.742      0.632      0.765      0.672
                    2h          8         10      0.708        0.6      0.751      0.606
                    2s         17         19          1      0.552      0.774      0.669
                    3c         13         17      0.912      0.612      0.799      0.669
                    3d         19         20      0.888        0.7      0.767        0.7
                    3

In [ ]:
from ultralytics import YOLO

# Load the trained model
model = YOLO("/content/runs/detect/train3/weights/best.pt")


In [ ]:
results = model("/content/drive/MyDrive/blackjack_dataset/blackjack/Capture11.JPG", save=True, conf=0.5)



image 1/1 /content/drive/MyDrive/blackjack_dataset/blackjack/Capture11.JPG: 384x640 1 wager-slot, 26.6ms
Speed: 3.0ms preprocess, 26.6ms inference, 3.3ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict
